In [ ]:
# 匯入套件
from bs4 import BeautifulSoup
import requests, json, re

In [ ]:
# 建立 list 來放置小說連結
book_links = []

# 標頭，沒有的話有些網站會將你認為是機器人、導致無法讀取
# 如何找標頭: 開發人員工具-->Network，重新整理、找第一個出來的物件，裡面最下面有一個user-agent
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

def getBooksLinks():
    # 金庸小說網站
    url = "https://www.bookwormzz.com/zh/"

    # 用 requests 的 get 方法把網頁抓下來
    response = requests.get(url, headers = headers)

    # 指定 lxml 作為解析器
    soup = BeautifulSoup(response.text, "lxml")

    books = soup.select('div.epub li a')

    for book in books:
        #print(book.get_text())
        #print("https://www.bookwormzz.com/" + book["href"][3:])
        book_links.append({
            "book": book.get_text(),
            "link": "https://www.bookwormzz.com/" + book["href"][3:]})
        
chapter_links = []
def getChapterLinks():

    for index, bookDict in enumerate(book_links):   
        #print("getChapterLinks:", bookDict["book"])
        res = requests.get(bookDict["link"])
        soup_link = BeautifulSoup(res.text, "lxml")
        #print("="*30)
        
        chapters = soup_link.select('#book_toc > div:nth-child(2) > div > ul > li a')
        #print(chapters)
    
        for chapter in chapters:
            #print(chapter)
            #print(chapter.get_text())
            #print("https://www.bookwormzz.com/zh" + chapter["href"])
            chapter_links.append({
                "book": bookDict["book"],
                "chapter_title": chapter.get_text(),
                "chapter_link": "https://www.bookwormzz.com/zh" + chapter["href"]
            })
    
        #book_links[index]["chapters"] = chapter_links
    
def getChapterContents():
    print("="*60)
    for index, chapterDict in enumerate(chapter_links):
        print("getChapterContents:", chapterDict["book"], chapterDict["chapter_title"])
        res = requests.get(chapterDict["chapter_link"])
        soup_link = BeautifulSoup(res.text, "lxml")
        texts = soup_link.select("#html > div:nth-child(1)")
        #print(texts[0].get_text())
        contents = re.sub(r"\s+", "",texts[0].get_text()) # 去除空白
        #print(contents)
        chapter_links[index]["contents"] = contents

# 將所有資訊轉成 JSON 檔
def saveJson():
    fp = open(".\\JingYong\\JingYong.json", "w", encoding = "utf-8")
    fp.write(json.dumps(chapter_links, ensure_ascii = False))
    fp.close()

# save to txt
def saveTxt():
    print("="*60)
    for index, chapterDict in enumerate(chapter_links):
        filename = ".\\JingYong\\" + chapterDict["book"] + "_" + chapterDict["chapter_title"] + ".txt"
        print("saveTxt:", filename)
        fp = open(filename, "w", encoding = "utf-8")
        fp.write(chapterDict["contents"])
        fp.close()
    #print("done")
    
# 主程式區段
if __name__ == "__main__":
    getBooksLinks()
    getChapterLinks()
    getChapterContents()
    saveJson()
    saveTxt()
    print("---------done----------")
    #print("-"*30)
    #print(book_links)

In [ ]:
# 測試，找每本書每章節的連結

res = requests.get(book_links[0])
soup_link = BeautifulSoup(res.text, "lxml")

for a in soup_link.select('a'):
    if a.has_attr('href'):
        print(a.get_text())
        print(a['href'])  #或寫 a.get("href") 也可以
        print("=" * 50)
    else:
        print()
        print("=" * 50)
        print(f"連結[{a.get_text()}] 沒有 href 屬性")
        print("=" * 50)
        print()